### 图的实现

In [3]:
#simpleGraph.py

class Node(object):
    def __init__(self, name):
        self.name = str(name)
    def getName(self):
        return self.name
    def __str__(self):
        return self.name

class Edge(object):
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return str(self.src) + '->' + str(self.dest)

class WeightedEdge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        self.src = src
        self.dest = dest
        self.weight = weight
    def getWeight(self):
        return self.weight
    def __str__(self):
        return str(self.src) + '->(' + str(self.weight) + ')'\
            + str(self.dest)

class Digraph(object):
    def __init__(self):
        self.nodes = set([])
        self.edges = {}
    def addNode(self, node):
        if node in self.nodes:
            raise ValueError('Duplicate node')
        else:
            self.nodes.add(node)
            self.edges[node] = []
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
    def childrenOf(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.nodes
    def __str__(self):
        res = ''
        for k in self.edges:
            for d in self.edges[k]:
                res = res + str(k) + '->' + str(d) + '\n'
        return res[:-1]

class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)


def printPath(path):
    # a path is a list of nodes
    result = ''
    for i in range(len(path)):
        if i == len(path) - 1:
            result = result + str(path[i])
        else:
            result = result + str(path[i]) + '->'
    return result

### L9 Problem 2

考虑以下问题：

- Alice, Bob和Carol站成一行；
- 每个顶点是三人的一个排列；
- 若两个排列能通过交换两个相邻学生得到，则两个顶点被边连起来

要求为图加上合适的边。

In [85]:
nodes = []
nodes.append(Node("ABC")) # nodes[0]
nodes.append(Node("ACB")) # nodes[1]
nodes.append(Node("BAC")) # nodes[2]
nodes.append(Node("BCA")) # nodes[3]
nodes.append(Node("CAB")) # nodes[4]
nodes.append(Node("CBA")) # nodes[5]

g = Graph()
for n in nodes:
    g.addNode(n)

加上合适的边。

In [86]:
d = {}

d['ABC'] = ['BAC', 'ACB']
d['ACB'] = ['CAB', 'ABC']
d['BAC'] = ['ABC', 'BCA']
d['BCA'] = ['CBA', 'BAC']
d['CAB'] = ['ACB', 'CBA']
d['CBA'] = ['BCA', 'CAB']

for i in range(6):
    nodeName = nodes[i].getName()
    #print 'From %s\n' % nodeName
    for j in [k for k in range(5) if k != i]:
        otherNodeName = nodes[j].getName()
        if otherNodeName in d[nodeName]:
            #print 'To %s which has children %s'%(otherNodeName, [n.getName() for n in g.childrenOf(nodes[j])])
            if nodeName not in [n.getName() for n in g.childrenOf(nodes[j])]:
                print 'Add: %s <-> %s\n' % (nodeName, nodes[j].getName())
                edgeToAdd = Edge(nodes[i], nodes[j])
                g.addEdge(edgeToAdd)

Add: ABC <-> ACB

Add: ABC <-> BAC

Add: ACB <-> CAB

Add: BAC <-> BCA

Add: CBA <-> BCA

Add: CBA <-> CAB



### 深度优先搜索

In [2]:
from simpleGraph import *

def DFS(graph, start, end, path = [], shortest = None):
    #assumes graph is a Digraph
    #assumes start and end are nodes in graph
    path = path + [start]
    print 'Current dfs path:', printPath(path)
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: #avoid cycles
            newPath = DFS(graph,node,end,path,shortest)
            if newPath != None:
                return newPath

def testSP():
    nodes = []
    for name in range(6):
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[1],nodes[2]))
    g.addEdge(Edge(nodes[2],nodes[3]))
    g.addEdge(Edge(nodes[2],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[5]))
    g.addEdge(Edge(nodes[0],nodes[2]))
    g.addEdge(Edge(nodes[1],nodes[0]))
    g.addEdge(Edge(nodes[3],nodes[1]))
    g.addEdge(Edge(nodes[4],nodes[0]))
    sp = DFS(g, nodes[0], nodes[5])
    print 'Shortest path found by DFS:', printPath(sp)

In [3]:
testSP()

Current dfs path: 0
Current dfs path: 0->1
Current dfs path: 0->1->2
Current dfs path: 0->1->2->3
Current dfs path: 0->1->2->3->4
Current dfs path: 0->1->2->3->5
Shortest path found by DFS: 0->1->2->3->5


假定有如下连接设定：

- [0]<->[1,2]
- [1]<->[2,0]
- [2]<->[3,4]
- [3]<->[4,5,1]
- [4]<->[0]
- [5]<->[ ]

简要分析：

- 0
- 0 -> 1
- 0 -> 1 -> 2
- 0 -> 1 -> 2 -> 3
- 0 -> 1 -> 2 -> 3 -> 4 (无路可走，因4的next节点0已被走过)
- 0 -> 1 -> 2 -> 3 -> 5 (回到节点3，走下一可能节点5，到达目的)

可以看出，`DFS`函数只是通过深度优先原则，取出第一条可能路径。而我们很快看到，`DFSShortest`才能得到我想要的结果。

In [14]:
def DFSShortest(graph, start, end, path = [], shortest = None):
    #assumes graph is a Digraph
    #assumes start and end are nodes in graph
    path = path + [start]
    print "================================"
    print 'Current dfs path:', printPath(path)
    if shortest != None:
        print "Current shortest path", printPath(shortest)
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: #avoid cycles
            if shortest == None or len(path)<len(shortest):
                newPath = DFSShortest(graph,node,end,path,shortest)
                if newPath != None and newPath != shortest:
                    #找到的newPath很可能还是现在的shortestPath，毕竟上一行中DFSShortest函数的返回值就是shortest
                    #因此，感觉有点低效，应该有提升空间
                    shortest = newPath
    return shortest

In [15]:
def testSP1():
    nodes = []
    for name in range(6):
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[1],nodes[2]))
    g.addEdge(Edge(nodes[2],nodes[3]))
    g.addEdge(Edge(nodes[2],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[5]))
    g.addEdge(Edge(nodes[0],nodes[2]))
    g.addEdge(Edge(nodes[1],nodes[0]))
    g.addEdge(Edge(nodes[3],nodes[1]))
    g.addEdge(Edge(nodes[4],nodes[0]))
    sp = DFSShortest(g, nodes[0], nodes[5])
    print 'Shortest path found by DFS:', printPath(sp)

testSP1()

Current dfs path: 0
Current dfs path: 0->1
Current dfs path: 0->1->2
Current dfs path: 0->1->2->3
Current dfs path: 0->1->2->3->4
Current dfs path: 0->1->2->3->5
Current dfs path: 0->1->2->4
Current shortest path 0->1->2->3->5
Current dfs path: 0->2
Current shortest path 0->1->2->3->5
Current dfs path: 0->2->3
Current shortest path 0->1->2->3->5
Current dfs path: 0->2->3->4
Current shortest path 0->1->2->3->5
Current dfs path: 0->2->3->5
Current shortest path 0->1->2->3->5
Current dfs path: 0->2->3->1
Current shortest path 0->2->3->5
Current dfs path: 0->2->4
Current shortest path 0->2->3->5
Shortest path found by DFS: 0->2->3->5


过程分析：

- (0, 5, [], None)
- (1, 5, [0], None)
- (2, 5, [0, 1], None)
- (3, 5, [0, 1, 2], None)
- (4, 5, [0, 1, 2, 3], None)=>走不动，回退
- (5, 5, [0, 1, 2, 3], None)=>更新shortest
- (4, 5, [0, 1, 2], [0, 1, 2, 3, 5])=>走不动，回退
- (2, 5, [0], [0, 1, 2, 3, 5])
- (3, 5, [0, 2], [0, 1, 2, 3, 5])
- (4, 5, [0, 2, 3], [0, 1, 2, 3, 5])=>走不动，回退
- (5, 5, [0, 2, 3], [0, 1, 2, 3, 5])=>更新shortest

求出所有从起点到终点的路径，作为对编程作业的帮助。先创建如上的图。

In [16]:
nodes = []
for name in range(6):
    nodes.append(Node(str(name)))
g = Digraph()
for n in nodes:
    g.addNode(n)
g.addEdge(Edge(nodes[0],nodes[1]))
g.addEdge(Edge(nodes[1],nodes[2]))
g.addEdge(Edge(nodes[2],nodes[3]))
g.addEdge(Edge(nodes[2],nodes[4]))
g.addEdge(Edge(nodes[3],nodes[4]))
g.addEdge(Edge(nodes[3],nodes[5]))
g.addEdge(Edge(nodes[0],nodes[2]))
g.addEdge(Edge(nodes[1],nodes[0]))
g.addEdge(Edge(nodes[3],nodes[1]))
g.addEdge(Edge(nodes[4],nodes[0]))

In [17]:
paths = []

def getPath(graph, start, end, path = []):
    path = path + [start]
    #print 'Current dfs path:', printPath(path)
    if start == end:
        paths.append(path)
    for node in graph.childrenOf(start):
        if node not in path: 
            getPath(graph, node, end, path)

getPath(g, nodes[0], nodes[5])

for path in paths:
    print printPath(path)

0->1->2->3->5
0->2->3->5


### 宽度优先搜索


In [146]:
from simpleGraph import *

def BFS(graph, start, end, q = []):
    initPath = [start]
    q.append(initPath)
    while len(q) != 0:
        tmpPath = q.pop(0)
        lastNode = tmpPath[len(tmpPath) - 1]
        print 'Current dequeued path:', printPath(tmpPath)
        if lastNode == end:
            return tmpPath
        for linkNode in graph.childrenOf(lastNode):
            if linkNode not in tmpPath:
                newPath = tmpPath + [linkNode]
                q.append(newPath)
    return None

假定有如下连接设定：

- [0]<->[1,2]
- [1]<->[2,0]
- [2]<->[3,4]
- [3]<->[4,5,1]
- [4]<->[0]
- [5]<->[ ]

实时跟踪双向队列的情况：

- [ [0], ]
- [ [0,1], [0,2] ]
- [ [0,1,2], [0,2,3], [0,2,4] ]
- [ [0,1,2,3], [0,1,2,4], [0,2,3,4], [0,2,3,5] ]

下面用程序验证我的分析。

In [147]:
def test():
    nodes = []
    for name in range(6):
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[1],nodes[0]))
    g.addEdge(Edge(nodes[2],nodes[3]))
    g.addEdge(Edge(nodes[2],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[4]))
    g.addEdge(Edge(nodes[3],nodes[5]))
    g.addEdge(Edge(nodes[0],nodes[2]))
    g.addEdge(Edge(nodes[1],nodes[2]))
    g.addEdge(Edge(nodes[3],nodes[1]))
    g.addEdge(Edge(nodes[4],nodes[0]))
    sp = BFS(g, nodes[0], nodes[5])
    print 'Shortest path found by DFS:', printPath(sp)
    
test()

Current dequeued path: 0
Current dequeued path: 0->1
Current dequeued path: 0->2
Current dequeued path: 0->1->2
Current dequeued path: 0->2->3
Current dequeued path: 0->2->4
Current dequeued path: 0->1->2->3
Current dequeued path: 0->1->2->4
Current dequeued path: 0->2->3->4
Current dequeued path: 0->2->3->5
Shortest path found by DFS: 0->2->3->5


### L9 Problem 7

In [ ]:
class WeightedEdge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        self.src = src
        self.dest = dest
        self.weight = weight
    def getWeight(self):
        return self.weight
    def __str__(self):
        return str(self.src) + '->' + str(self.dest) + ' (' + str(self.weight) + ')'